# Generate point-in-time best execution data using dedicated service - Python

### Overview
Best execution aims to measure the performance of executed trades, an execution algorithm or an execution venue.

This sample is based on an `point-in-time` approach and designed to highlight individual trades within market activity over a look back period. 

<div class="alert alert-block alert-info">
    <b>Note:</b> To explore the interval approach, suitable to analyze market activity over a look back period. Please refer to <i>bestex-interval</i> notebooks.
</div>

This sample demonstrates how to request and plot from a dedicated data service **on-demand** best execution *point-in-time* results.

### Inputs/outputs
Best execution PIT sample requires instrument's identifier, **single trade** and time window as per inputs. Results are as follows:
* a snapshot of occured trades around the given trade based on the input time window

### Services used
This sample uses *gRPC requests* in order to retrieve trades information from the hosted service. The queried endpoint in this script are:
* *TickTradesService*: to directly retrieve trades data from the server.

### Modules required
1. Systemathics:
    * *systemathics.apis.services.tick.v1*
    * *systemathics.apis.type.shared.v1*
    * *google.type*
2. Open source:
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    * *matpotlib* as per display package
    
***


# Run BestEx Point-in-time sample

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas matplotlib plotly

In [ ]:
pip install systemathics.apis==2.38.* --pre

In [ ]:
import os
import grpc
import pandas as pd
from datetime import datetime
from datetime import timedelta
import google.type.date_pb2 as date
import google.type.timeofday_pb2 as timeofday
import google.type.dayofweek_pb2 as dayofweek
import google.protobuf.duration_pb2 as duration
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.type.shared.v1.constraints_pb2 as constraints
import systemathics.apis.type.shared.v1.date_interval_pb2 as dateinterval
import systemathics.apis.type.shared.v1.time_interval_pb2 as timeinterval
import systemathics.apis.services.tick.v1.tick_trades_pb2 as tick_trades
import systemathics.apis.services.tick.v1.tick_trades_pb2_grpc as tick_trades_service
import systemathics.apis.helpers.token_helpers as token_helpers
import systemathics.apis.helpers.channel_helpers as channel_helpers

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = token_helpers.get_token()
display(token)

### Step 3: Create and process request

#### 3.1 Instrument selection

In [ ]:
# set instrument identifier: exchange + ticker
ticker = 'AAPL'
exchange = 'BATS'

# set the time window to delimit the input trade
window = 15

#### 3.2 Trade parameters

We assume our trade is the following:

In [ ]:
timestamp = datetime(2021, 11, 5, 12, 25, 39)
micros = 469200
price = 151.05
size = 13

#### 3.2 Time period delimitation

In [ ]:
# create date interval (we are using Google date format)
date_interval = dateinterval.DateInterval(
    start_date = date.Date(year = timestamp.year, month = timestamp.month, day = timestamp.day), 
    end_date = date.Date(year = timestamp.year, month = timestamp.month, day = timestamp.day)
)

# create time interval (we are using Google time format)
# UTC time zone
start = timestamp - timedelta(minutes = window)
end = timestamp + timedelta(minutes = window)
time_interval = timeinterval.TimeInterval(
    start_time = timeofday.TimeOfDay(hours = start.hour, minutes = start.minute, seconds = start.second), 
    end_time = timeofday.TimeOfDay(hours = end.hour, minutes = end.minute, seconds = end.second)
)

#### 3.3 Request creation
The following code snippet creates *gRPC client*, process request and returns the request reply:

In [ ]:
# generate constraints based on the previous time selection
my_constraints = constraints.Constraints(
    date_intervals = [date_interval],
    time_intervals = [time_interval],
)

In [ ]:
# generate the tick trades request
my_identifier = identifier.Identifier(exchange = exchange, ticker = ticker)
request = tick_trades.TickTradesRequest(
    identifiers = [my_identifier],
    constraints = my_constraints
)

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  

        # instantiate the tick trades service
        service = tick_trades_service.TickTradesServiceStub(channel)
        
        # process the tick trades request
        trades = []
        metadata = [('authorization', token)]
        for trade in service.TickTrades(request=request, metadata=metadata):
            if trade.data:
                trades.append(trade)

    # keep only trades, delete instrument mapping (1st element retrieved)
    # mapping is not used as only one instrument identifier is requested
    del trades[0]
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

The following code snippet displays the total retrieved trades count:

In [ ]:
print('Total trades retrieved: {0}'.format(len(trades)))

### Step 4: Visualize data

#### 4.1 Retrieve market data
In the following code snippets, the reply is reprocessed in a data frame in order to visualize the results with ease:

In [ ]:
# prepare the dataframe content
dates = [(datetime.fromtimestamp(t.data.time_stamp.seconds) + timedelta(microseconds = t.data.time_stamp.nanos/1000)) for t in trades]
prices = [t.data.trade.price for t in trades]
sizes = [t.data.trade.size for t in trades]

In [ ]:
# create a pandas dataframe with: dates, trades prices and sizes
d = {'Date': dates, 'Price': prices, 'Size': sizes}
df = pd.DataFrame(data=d)
df

#### 4.2 Retrieve input trade

In [ ]:
# user input timestamp is split into datetime and microseconds input to preserve trade accuracy
timestamp += timedelta(microseconds = micros)

#### 4.3 Plot trades with matplotlib

In [ ]:
import matplotlib.pyplot as plt

fig,ax = plt.subplots(1,1,figsize=(25,10))
# add price plots
ax.plot('Date', 'Price', data=df, marker='', color='blue', alpha =0.25, linewidth=2, label="Price")
ax.plot(timestamp, price, color='blue', linestyle='None', marker='D', markersize=10, label="Input trade price")

# add twin x-axis for size plot
ax2=ax.twinx()
ax2.plot('Date', 'Size', data=df, marker='', color='red', alpha =0.25, linewidth=2, label="Size")
ax2.plot(timestamp, size, color='red', linestyle='None', marker='D', markersize=10, label="Input trade size")

# set graph layout
ax.set_xlabel("Date",fontsize=14)
ax.set_ylabel("Price",color="blue",fontsize=14)
ax2.set_ylabel("Size",color="red",fontsize=14)
plt.title('Input trade within market snapshot for {0}-{1}'.format(ticker, exchange))
ax.legend(loc='upper left')
plt.legend(loc='upper right')
plt.show()